In [1]:
# %pip install pandas
import pandas as pd
from sklearn.preprocessing  import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv(r'C:\\Users\\Kaybee\\Videos\\Internship_hamoyeHQ\\NFA 2019 public_data.csv')
df.head(100)
df[df['country'] == 'Solomon Islands']

C:\Users\Kaybee\AppData\Local\Temp\ipykernel_9476\320100654.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\\Users\\Kaybee\\Videos\\Internship_hamoyeHQ\\NFA 2019 public_data.csv')


,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
7424,Solomon Islands,2016,25,AreaPerCap,0.166828,0.013346,3.635854014,5.616939e+00,0.063014,0.000000,9.495981e+00,1A
7425,Solomon Islands,2016,25,AreaTotHA,100000.000000,8000.000000,2179400,3.366900e+06,37771.598820,0.000000,5.692072e+06,1A
7426,Solomon Islands,2016,25,BiocapPerCap,0.648189,0.006129,2.03713899,1.109850e+00,0.244831,0.000000,4.046137e+00,1A
7427,Solomon Islands,2016,25,BiocapTotGHA,388536.624200,3673.652281,1221099.829,6.652650e+05,146756.494900,0.000000,2.425332e+06,1A
7428,Solomon Islands,2016,25,EFConsPerCap,0.591631,0.023550,0.135055655,1.213959e+00,0.244831,0.217236,2.426263e+00,1A
7429,Solomon Islands,2016,25,EFConsTotGHA,354635.095500,14116.340470,80954.92673,7.276700e+05,146756.494900,130215.502300,1.454348e+06,1A
7430,Solomon Islands,2016,25,EFProdPerCap,0.648189,0.006129,3.209020706,2.627408e+00,0.244831,0.123195,6.858772e+00,1A
7431,Solomon Islands,2016,25,EFProdTotGHA,388536.624200,3673.652281,1923548.003,1.574918e+06,146756.494900,73845.330570,4.111278e+06,1A


In [3]:
# dropping all empty rows
df['QScore'].value_counts()
df.isnull().sum()
df = df.dropna()
df.isna().sum()


country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

#### An obvious change in our target variable after removing the missing values is that there are only three classes left #and from the distribution of the 3 classes, we can see that there is an obvious imbalance between the classes. There are methods that can be applied to handle this imbalance such as oversampling and undersampling.
#### Oversampling involves increasing the number of instances in the class with fewer instances while undersampling involves reducing the data points in the class with more instances.
#### For now, we will convert this to a binary classification problem by combining class '2A' and '1A'.

In [4]:
df["QScore"].value_counts()
df["QScore"] = df["QScore"].replace(['1A'], '2A')
df["QScore"].value_counts()

QScore
3A    51473
2A      240
Name: count, dtype: int64

In [5]:
df_2A = df[df.QScore == '2A']
df_3A = df[df.QScore == '3A'].sample(350)
df_append =  pd.concat([df_2A, df_3A], ignore_index= True)
df_append['QScore']

0      2A
1      2A
2      2A
3      2A
4      2A
       ..
585    3A
586    3A
587    3A
588    3A
589    3A
Name: QScore, Length: 590, dtype: object

In [6]:
# from sklearn import
# !pip install scikit-learn

In [7]:
# from sklearn import sklearn.utils
import sklearn.utils
df_append = sklearn.utils.shuffle(df_append)
df_append = df_append.reset_index(drop = True)
df_append.shape
df_append["QScore"].value_counts()

QScore
3A    350
2A    240
Name: count, dtype: int64

In [8]:
df_append = df_append.drop(columns= ['country_code', 'year'])
df_append
X = df_append.drop(columns= ['QScore'])
y = df_append['QScore']


In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
y_train.value_counts()

QScore
3A    248
2A    165
Name: count, dtype: int64

In [10]:
# There is imbalance in the class distribution
# Encode categorical
encoder = LabelEncoder()
x_train.record = encoder.fit_transform(x_train.record)
x_test.record = encoder.transform(x_test.record)


In [11]:
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state  = 1)
x_train_balanced, y_balanced = smote.fit_resample(x_train, y_train)
scaler = MinMaxScaler()
normalised_df = scaler.fit_transform(x_train_balanced.drop(columns = ['record']))
normalised_df = pd.DataFrame(normalised_df, columns = x_train_balanced.drop(columns= 'record').columns)
normalised_df['record'] = x_train_balanced['record']

x_test = x_test.reset_index(drop=True)
normalised_test_df = scaler.transform(x_test.drop(columns=['record']))
normalised_test_df = pd.DataFrame(normalised_test_df, columns=x_test.drop(columns=['record']).columns)
normalised_test_df['record'] = x_test['record']


#Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_df, y_balanced)
#returns
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


ValueError: could not convert string to float: 'Costa Rica'